# 4. Practical part: Density estimation

IFT 6390 Fall/Automne 2018, HW1, Q4, practical

Arlie Coles (20121051) & Violet Guo ()

*Note to TAs: these implementations run on Python 3.*

## 1. Implementation of a Gaussian parametric density estimator

In [4]:
import numpy as np
import math

class gaussianDiag:
    def __init__(self, data, label):
        self.train_data = data
        self.train_label = label
        self.train_mean = 0
        self.train_cov = 0
    
    def train_diag(self):
        '''
        Calculate the mean and generate a diagonal covariance matrix
        '''
        d = self.train_data.shape
        # print(d)
        self.train_mean = np.mean(self.train_data)
        print("The training mean is: {}".format(self.train_mean))
        self.train_cov = np.multiply( 
            (self.train_data - self.train_mean), np.transpose((self.train_data - self.train_mean))
        )
        self.train_cov =np.diag(self.train_cov /d)
        print("The training covariance matrix is: \n {}".format(self.train_cov))
        
        return self.train_mean, self.train_cov


    def predict_diag(self, train_label):
        # I'm not too sure what's going on here - Arlie
        '''
        Calculate the log density
        '''
    
        # Take the inverse of diagonals cov
        print("np.diag(self.train_cov)", np.diag(self.train_cov))
        train_cov_inv = np.linalg.inv(self.train_cov)
        print("train_cov_inv", train_cov_inv)
        
        


data = np.array([2 ,2,3])
label = np.array([3,2,2])
part1 = gaussianDiag(data, label)

part1.train_diag()
part1.predict_diag(label)

The training mean is: 2.33333333333
The training covariance matrix is: 
 [[ 0.03703704  0.          0.        ]
 [ 0.          0.03703704  0.        ]
 [ 0.          0.          0.14814815]]
('np.diag(self.train_cov)', array([ 0.03703704,  0.03703704,  0.14814815]))
('train_cov_inv', array([[ 27.  ,   0.  ,   0.  ],
       [  0.  ,  27.  ,   0.  ],
       [  0.  ,   0.  ,   6.75]]))


## 2:  Implementation of a Parzen density estimator

In [63]:
class parzenEstimator:
    def __init__(self, data, kernel_stdev):
        self.train_data = data
        self.kernel_stdev = kernel_stdev
        self.d = len(self.train_data.T) # Get dimension; shape method too finicky returning "L" for long int type

    def train(self, train_data):
        # Loading datapoints was taken care of in the constructor above.
        # This function creates the lambda function that makes summing easier/faster in the predict() function.
        # This metaphorically "creates" a Gaussian centered on each datapoint, while leaving the slot in the pdf for
        #  the test point open, until the weighting actually occurs at test time.
        norm_const = 1/np.multiply((2*math.pi)**(self.d/2), self.kernel_stdev**self.d)
        self.summand = lambda x, mu : np.multiply(norm_const, np.exp(np.multiply(-0.5, ((np.absolute(x - mu))/(self.kernel_stdev**2)))))

    def predict(self, test_point):
        # Sum over the kernels with the datapoint as x
        density = 0
        for datapoint in self.train_data:
            density += self.summand(test_point, datapoint)
        log_density = np.log(density)
        return log_density

We test the Parzen estimator with some dummy data:

In [64]:
data = np.array([[2 ,2,3], [2,2,2]])
test_point = np.array([1,2,3])

part2 = parzenEstimator(data, 0.5)
part2.train(data)
print(part2.predict(test_point))

[-1.06528834  0.93471166  0.36849249]


## 3. 1D densities

First, let's open and parse the Iris dataset, selecting just the versicolor class:

In [71]:
with open('iris.data', 'r') as fp:
    iris_data = fp.readlines()

# Get just the versicolor class
versicolor_lines = []
for line in iris_data:
    if 'versicolor' in line:
        versicolor_lines.append(line.strip())

Then, let's further select just the data for sepal length:

In [74]:
data_1d = []
for line in versicolor_lines:
    data_1d.append(line.split(',')[0])

### Part (a)

A plot of the data points of the subset:

In [78]:
import matplotlib.pyplot as plt

print(data_1d)
plt.plot(data_1d)
plt.xlabel()
plt.show()

['7.0', '6.4', '6.9', '5.5', '6.5', '5.7', '6.3', '4.9', '6.6', '5.2', '5.0', '5.9', '6.0', '6.1', '5.6', '6.7', '5.6', '5.8', '6.2', '5.6', '5.9', '6.1', '6.3', '6.1', '6.4', '6.6', '6.8', '6.7', '6.0', '5.7', '5.5', '5.5', '5.8', '6.0', '5.4', '6.0', '6.7', '6.3', '5.6', '5.5', '5.5', '6.1', '5.8', '5.0', '5.6', '5.7', '5.7', '6.2', '5.1', '5.7']


TypeError: xlabel() takes at least 1 argument (0 given)